## Title: Predicting Customer Churn - A Machine Learning Approach

Description: Customer attrition, also known as customer churn, is a critical challenge faced by businesses. The goal of this project is to develop a predictive model that can identify customers who are likely to churn, allowing the organization to implement targeted retention strategies and reduce customer churn rate. By understanding the key factors that influence customer churn, we aim to provide valuable insights that will help the company make informed decisions to improve customer retention and loyalty

In [39]:
#install required packages
%pip install pyodbc  
%pip install python-dotenv
%pip install pandas
%pip install sklearn
%pip install openpyxl
%pip install imblearn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import all necessary packages

In [3]:

import pyodbc #just installed with pip
from dotenv import dotenv_values #import the dotenv_values function from the dotenv package
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

import openpyxl
import warnings 

warnings.filterwarnings('ignore')

### Data Loading
First Data Set

In [4]:
# Load environment variables from .env file into a dictionary
environment_variables=dotenv_values('.env')

# Get the values for the credentials you set in the '.env' file
database=environment_variables.get("DATABASE")
server=environment_variables.get("SERVER")
username=environment_variables.get("USERNAME")
password=environment_variables.get("PASSWORD")

connection_string=f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"

In [5]:
# Use the connect method of the pyodbc library and pass in the connection string.
# This will connect to the server and might take a few seconds to be complete. 
# Check your internet connection if it takes more time than necessary

connection=pyodbc.connect(connection_string)

In [6]:
# Now the sql query to get the data is what what you see below. 
# Note that you will not have permissions to insert delete or update this database table. 

query="Select * from dbo.LP2_Telco_churn_first_3000"
data=pd.read_sql(query,connection)

In [7]:
#inspect the first five rows of the first data set
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,False,True,False,1,False,None,DSL,False,...,False,False,False,False,Month-to-month,True,Electronic check,29.850000,29.850000,False
1,5575-GNVDE,Male,False,False,False,34,True,False,DSL,True,...,True,False,False,False,One year,False,Mailed check,56.950001,1889.500000,False
2,3668-QPYBK,Male,False,False,False,2,True,False,DSL,True,...,False,False,False,False,Month-to-month,True,Mailed check,53.849998,108.150002,True
3,7795-CFOCW,Male,False,False,False,45,False,None,DSL,True,...,True,True,False,False,One year,False,Bank transfer (automatic),42.299999,1840.750000,False
4,9237-HQITU,Female,False,False,False,2,True,False,Fiber optic,False,...,False,False,False,False,Month-to-month,True,Electronic check,70.699997,151.649994,True


### Second Data Set

In [8]:
#Load the second dataframe and inspect the first five rows
data2=pd.read_csv('LP2_Telco-churn-last-2000.csv')
data2.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,5600-PDUJF,Male,0,No,No,6,Yes,No,DSL,No,...,No,Yes,No,No,Month-to-month,Yes,Credit card (automatic),49.50,312.7,No
1,8292-TYSPY,Male,0,No,No,19,Yes,No,DSL,No,...,Yes,Yes,No,No,Month-to-month,Yes,Credit card (automatic),55.00,1046.5,Yes
2,0567-XRHCU,Female,0,Yes,Yes,69,No,No phone service,DSL,Yes,...,Yes,No,No,Yes,Two year,Yes,Credit card (automatic),43.95,2960.1,No
3,1867-BDVFH,Male,0,Yes,Yes,11,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,74.35,834.2,Yes
4,2067-QYTCF,Female,0,Yes,No,64,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,111.15,6953.4,No


### Third Data Set

In [9]:
data3 = pd.read_excel("Telco-churn-second-2000.xlsx")
data3.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7613-LLQFO,Male,0,No,No,12,Yes,Yes,Fiber optic,No,No,No,No,Yes,No,Month-to-month,Yes,Electronic check,84.45,1059.55
1,4568-TTZRT,Male,0,No,No,9,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,20.40,181.8
2,9513-DXHDA,Male,0,No,No,27,Yes,No,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,No,Electronic check,81.70,2212.55
3,2640-PMGFL,Male,0,No,Yes,27,Yes,Yes,Fiber optic,No,No,No,Yes,No,No,Month-to-month,Yes,Electronic check,79.50,2180.55
4,3801-HMYNL,Male,0,Yes,Yes,1,Yes,No,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,No,Mailed check,89.15,89.15


In [10]:
# You can concatenate this with other DataFrames to get one data set for your work
# !!!Concatenation done for data data2 and data3
df = pd.concat([data,data2,data3])
df.to_csv('aba.csv')

## Exploratory Data Analysis

In the exploratory data analysis phase, we will perform univariate, bivariate, and multivariate analysis to gain insights into the data. Visualizations such as bar charts, histograms, scatter plots, and correlation matrices will be used to understand the distribution of variables, relationships between features, and their impact on customer churn.

In [11]:
data.shape

(3000, 21)

In [12]:
data2.shape

(2043, 21)

In [13]:
data3.shape

(2000, 20)

In [14]:
df.shape

(7043, 21)

In [15]:
#Check the shapes of the dataframes
df.head().T

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,False,False,False,False,False
Partner,True,False,False,False,False
Dependents,False,False,False,False,False
tenure,1,34,2,45,2
PhoneService,False,True,True,False,True
MultipleLines,None,False,False,None,False
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,False,True,True,True,False


## Hypothesis
Customers on month-to-month contracts are more likely to churn compared to those on one-year or two-year contracts.

**Questions:**

1. Is there a relationship between the type of internet service (DSL, Fiber Optic, No Internet) and customer churn?
2. Does the monthly charge amount impact customer churn rate? Are higher monthly charges associated with higher churn?
3. Do customers who have additional services like online security, tech support, etc., have lower churn rates?
4. Is there a correlation between the tenure (number of months a customer has stayed with the company) and the likelihood of churn? Do customers who have been with the company for a longer time exhibit lower churn rates?
5. How does the payment method influence customer churn? Are customers using automatic payment methods (Electronic check, Bank transfer(automatic), Credit card(automatic)) less likely to churn compared to those using manual methods (mailed check)?

## Potential Data Issues with attempted solutions

1. Missing Values: We will check for missing values in the dataset and decide how to handle them. If there are only a few missing values, we may choose to drop those rows. If a significant number of records have missing values, we can consider imputation techniques like mean, median, or mode.

2. Data Types: We will ensure that the data types of each column are appropriate for the analysis. Categorical variables should be encoded as numeric values, and continuous variables should remain as numeric.

3. Class Imbalance: We need to check for class imbalance in the target variable (Churn). If there is a severe class imbalance, we may need to address it using techniques such as oversampling, undersampling, or using appropriate evaluation metrics.

4. Feature Scaling: Some machine learning algorithms may require feature scaling to ensure that all features contribute equally to the model. We will scale the numerical features if necessary.

5. Handling Categorical Variables: We will use one-hot encoding to convert categorical variables into a binary form suitable for model training.

6. Data Splitting: Before model training, we will split the data into training and testing sets to evaluate the model's performance on unseen data.

By addressing these issues during data preprocessing, we can ensure that our dataset is ready for model building and analysis.



### Important terminologies:
Classifier: An algorithim that is used to map the input data to a specific category.

Classification model: The model that predicts the input data given for training.

Feature: It is an individual measurable property of the phenomenon being observed.

Labels: The characteristics on which the datapoints of a dataset  are categorized. 

In [16]:
# We start with Data Types
df.dtypes

customerID           object
gender               object
SeniorCitizen        object
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [17]:
# We expect Total Charges column be numeric, as it contains the total amount of money the client was charged/ 
# so it should not be an object.
total_charges = pd.to_numeric(df.TotalCharges, errors='coerce')

In [18]:
# Currently, the 'Churn' column is categorical, with two values, “yes” and “no”. For binary classification, \n 
# all models typically expect a number: 0 for “no” and 1 for “yes.” Let’s convert it to numbers.

df.Churn = (df.Churn == 'yes').astype(int)

In [19]:
# Missing Values
df.isnull().sum()

customerID            0
gender                0
SeniorCitizen         0
Partner               0
Dependents            0
tenure                0
PhoneService          0
MultipleLines       269
InternetService       0
OnlineSecurity      651
OnlineBackup        651
DeviceProtection    651
TechSupport         651
StreamingTV         651
StreamingMovies     651
Contract              0
PaperlessBilling      0
PaymentMethod         0
MonthlyCharges        0
TotalCharges          5
Churn                 0
dtype: int64

In [20]:
df['Churn'].dtype

dtype('int32')

In [21]:
#Let's start filling in missing values.
# From the above TotalCharges column contains missing values of 5. We fill missing values with 0.
df.TotalCharges = df.TotalCharges.fillna(0)

In [22]:
# Missing Values
df.isnull().sum()

customerID            0
gender                0
SeniorCitizen         0
Partner               0
Dependents            0
tenure                0
PhoneService          0
MultipleLines       269
InternetService       0
OnlineSecurity      651
OnlineBackup        651
DeviceProtection    651
TechSupport         651
StreamingTV         651
StreamingMovies     651
Contract              0
PaperlessBilling      0
PaymentMethod         0
MonthlyCharges        0
TotalCharges          0
Churn                 0
dtype: int64